In [1]:
import requests
import folium

def plot_osrm_route(locations, map_filename='route_map.html'):
    """
    locations: list of [lat, lon]
    map_filename: output HTML file
    """
    # Convert to lon,lat order for OSRM
    coords_str = ";".join([f"{lon},{lat}" for lat, lon in locations])

    # Request route from OSRM public server
    url = f"http://router.project-osrm.org/route/v1/driving/{coords_str}"
    params = {
        "overview": "full",     # get full geometry
        "geometries": "geojson" # geojson geometry
    }

    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()

    route_coords = data['routes'][0]['geometry']['coordinates']  # list of [lon,lat]
    distance_km = data['routes'][0]['distance'] / 1000
    duration_min = data['routes'][0]['duration'] / 60

    print(f"Route distance: {distance_km:.2f} km")
    print(f"Route duration: {duration_min:.1f} min")

    # Create folium map centered on first location
    start_lat, start_lon = locations[0]
    fmap = folium.Map(location=[start_lat, start_lon], zoom_start=13)

    # Add route line
    route_latlon = [[lat, lon] for lon, lat in route_coords]  # convert to [lat,lon]
    folium.PolyLine(route_latlon, color='blue', weight=5, opacity=0.8).add_to(fmap)

    # Add markers
    folium.Marker([start_lat, start_lon], popup="Start", icon=folium.Icon(color='green')).add_to(fmap)
    end_lat, end_lon = locations[-1]
    folium.Marker([end_lat, end_lon], popup="End", icon=folium.Icon(color='red')).add_to(fmap)

    # Optional: intermediate points
    for lat, lon in locations[1:-1]:
        folium.CircleMarker([lat, lon], radius=4, color='gray', fill=True).add_to(fmap)

    # Save to HTML
    fmap.save(map_filename)
    print(f"Map saved to {map_filename}")
    return fmap


# Example usage:
locations = [
    [1.290270, 103.851959],  # Marina Bay, Singapore
    [1.352083, 103.819839],  # Another point
    [1.282302, 103.819836]   # Another point
]

plot_osrm_route(locations)


Route distance: 25.13 km
Route duration: 39.6 min
Map saved to route_map.html


## Start running here

In [13]:
import requests
import pandas as pd

def osrm_distance_matrix(locations):
    """
    locations: list of [lat, lon]
    returns: pandas DataFrame of pairwise distances in km
    """
    # Convert to lon,lat for OSRM
    coords_str = ";".join([f"{lon},{lat}" for lat, lon in locations])

    # Request distance matrix from OSRM table service
    url = f"http://router.project-osrm.org/table/v1/driving/{coords_str}"
    params = {
        "annotations": "distance"  # return distances
    }

    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()

    # distances in meters
    distances_m = data['distances']  # square matrix
    distances_km = [[round(m/1000, 2) for m in row] for row in distances_m]

    # Create DataFrame
    labels = [f"P{i+1}" for i in range(len(locations))]
    print(distances_m)
    df = pd.DataFrame(distances_m, columns=labels, index=labels)

    return df

# Example usage
locations = [
    [1.290270, 103.851959],  # Marina Bay
    [1.352083, 103.819839],  # Somewhere north
    [1.282302, 103.819836]   # Somewhere west
]

df = osrm_distance_matrix(locations)
print(df)

df.to_csv("distance_matrix.csv")
print("Saved distance matrix to distance_matrix.csv")


[[0, 10780.5, 5240.5], [11491.8, 0, 14331.8], [6033.8, 15694.4, 0]]
         P1       P2       P3
P1      0.0  10780.5   5240.5
P2  11491.8      0.0  14331.8
P3   6033.8  15694.4      0.0
Saved distance matrix to distance_matrix.csv


# Try creating a pairwise distance matrix from OSRM with my dummy locations

In [2]:
# My locations
my_locations = pd.read_csv("data/locations_w_node_id.csv")
my_locations.head()

,id,location_name,location_type,lat,lon,node_id
0,school_1,Admiralty Primary School,school,1.44294,103.80035,8655379195
1,school_2,Jurong West Primary School,school,1.33931,103.69903,7437327501
2,school_3,River Valley Primary School,school,1.29430,103.83593,11938018645
3,school_4,Casuarina Primary School,school,1.37281,103.95723,8302208139
4,school_5,St. Joseph’s Institution Junior,school,1.31751,103.84632,10979316519


In [3]:
# Create list of lat lons
lat_list = list(my_locations['lat'])
lon_list = list(my_locations['lon'])

location_coordinates = [[lat, lon] for lat, lon in zip(lat_list, lon_list)]

In [4]:
osrm_pairwise_distance_matrix = osrm_distance_matrix(location_coordinates)
osrm_pairwise_distance_matrix

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31
P1,0.0,21540.1,25038.5,25018.8,22000.5,24032.3,13657.3,17329.5,4100.0,20732.2,...,27389.9,12562.9,10195.8,21956.2,30747.7,23200.9,17687.4,14476.5,20471.7,25812.2
P2,21214.5,0.0,20892.3,37369.7,20995.4,26298.9,21792.1,26889.6,18670.3,11282.2,...,20857.8,22339.9,26497.1,10684.3,18868.5,16614.9,16020.3,9570.7,11021.7,6450.1
P3,25144.2,22520.0,0.0,22545.9,4706.0,7585.6,11353.2,13277.1,20804.6,10219.8,...,2085.9,11334.6,13289.2,12550.3,5723.4,4977.1,6066.3,17688.9,9815.7,19495.4
P4,25169.5,38443.5,22194.6,0.0,19156.6,14034.4,17044.1,12341.7,26507.7,31814.3,...,24546.0,15962.7,20831.8,33616.2,27903.8,27232.0,22934.0,32758.4,31553.8,40561.2
P5,21783.9,20860.4,4134.0,19185.6,0.0,5707.9,6548.6,9916.8,19074.8,14231.2,...,5223.8,7974.4,7580.9,16868.1,11155.7,10483.9,5350.9,15959.1,13970.7,23813.1
P6,22539.4,26280.0,8047.9,14617.2,5471.8,0.0,8465.5,7916.4,23877.6,19650.8,...,8134.8,8729.8,10684.5,17926.7,11246.3,11542.5,10770.4,21378.7,19390.2,24871.7
P7,13640.0,22719.9,11366.3,17565.5,6607.8,10360.1,0.0,6541.7,14978.2,16090.6,...,13717.7,2530.7,3908.7,16858.7,17075.5,12519.4,7005.8,17818.5,14478.0,23803.7
P8,17773.3,28111.0,13331.3,12891.6,10293.3,8281.6,6716.6,0.0,19111.5,21481.8,...,15682.7,5031.3,8163.6,24752.9,19040.5,18368.7,12397.0,23209.6,19869.1,31697.9
P9,4122.0,18462.2,20589.9,25452.3,19142.2,24445.7,14090.8,17763.0,0.0,17654.3,...,21466.0,12996.4,10629.4,18878.3,26288.8,21929.9,17334.3,11398.5,17393.8,22734.2
P10,19999.1,11655.7,11943.2,31379.9,12887.9,20309.1,15802.3,20899.8,17454.9,0.0,...,11908.7,16350.1,16698.8,3111.8,9919.3,7665.7,8189.1,10650.7,1111.3,9244.4


In [14]:
osrm_pairwise_distance_matrix.iloc[[1,5,7],[1,5,7]]

,P2,P6,P8
P2,0.0,26298.9,26889.6
P6,26280.0,0.0,7916.4
P8,28111.0,8281.6,0.0


In [6]:
import numpy as np
def pairwise_matrix_to_or_tools_format(pairwise_matrix):
    nested_list = (np.array(pairwise_matrix.values.tolist())).astype(int)
    return nested_list

In [7]:
pairwise_matrix_to_or_tools_format(osrm_pairwise_distance_matrix)
pairwise_matrix_to_or_tools_format(osrm_pairwise_distance_matrix.iloc[[1,5,7],[1,5,7]])

array([[    0, 26298, 26889],
       [26280,     0,  7916],
       [28111,  8281,     0]])

In [9]:
len(osrm_pairwise_distance_matrix.iloc[[1,5,7],[1,5,7]])

3

In [15]:
# next steps, create a pipeline to move the click events to distance matrix
"""Simple Vehicles Routing Problem (VRP).

This is a sample using the routing library python wrapper to solve a VRP
problem.
A description of the problem can be found here:
http://en.wikipedia.org/wiki/Vehicle_routing_problem.

Distances are in meters.
"""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [[    0, 17874.3, 30897.1],
                                [16997, 0, 29176.7],
                                [30830.7, 29294.5,0]]
    data["num_vehicles"] = min(3, len(osrm_pairwise_distance_matrix.iloc[[1,5,7],[1,5,7]]))
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        if not routing.IsVehicleUsed(solution, vehicle_id):
            continue
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")



def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        200000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    
    def get_routes(solution, routing, manager):
        """Get vehicle routes from a solution and store them in an array."""
        # Get vehicle routes and store them in a two dimensional array whose
        # i,j entry is the jth location visited by vehicle i along its route.
        routes = []
        for route_nbr in range(routing.vehicles()):
            index = routing.Start(route_nbr)
            route = [manager.IndexToNode(index)]
            while not routing.IsEnd(index):
                index = solution.Value(routing.NextVar(index))
                route.append(manager.IndexToNode(index))
            routes.append(route)
        return routes
    
    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
        routes = get_routes(solution, routing, manager)
        return routes
    else:
        print("No solution found !")


if __name__ == "__main__":
    routes = main()

Objective: 0
Route for vehicle 2:
 0 ->  2 ->  1 -> 0
Distance of the route: 0m

Maximum of the route distances: 0m


In [9]:
routes

[[0, 1, 0], [0, 0], [0, 2, 0]]

In [11]:
my_locations.head()

,id,location_name,location_type,lat,lon,node_id
0,school_1,Admiralty Primary School,school,1.44294,103.80035,8655379195
1,school_2,Jurong West Primary School,school,1.33931,103.69903,7437327501
2,school_3,River Valley Primary School,school,1.29430,103.83593,11938018645
3,school_4,Casuarina Primary School,school,1.37281,103.95723,8302208139
4,school_5,St. Joseph’s Institution Junior,school,1.31751,103.84632,10979316519


In [12]:
def routes_to_location_names(routes, location_df):
    """
    Shows routes as a list of location names

    Args:
        routes: a list of list, indices of the dataframe
        location_df: a dataframe with the col name 'location_name'
    """
    list_of_routes_as_location_names = []
    location_name_list = location_df['location_name']
    for route in routes:
        current_location_name_list = [location_name_list[idx] for idx in route]
        list_of_routes_as_location_names.append(current_location_name_list)
    return list_of_routes_as_location_names


In [13]:
routes_to_location_names(routes, my_locations)

[['Jurong Port',
  'Whitley Secondary School',
  'S11 Ang Mo Kio',
  'Xinmin Secondary School',
  'Waterway Point',
  'NEX',
  'St. Joseph’s Institution Junior',
  'ION Orchard',
  'Clementi 448 Market & Food Centre',
  'Jurong Port'],
 ['Jurong Port',
  'Ayer Rajah Food Centre',
  'White Sands',
  'Casuarina Primary School',
  'Geylang Methodist School Primary',
  'Jurong Port'],
 ['Jurong Port',
  'ABC Brickworks Market & Food Centre',
  'Seah Im Food Centre',
  'VivoCity',
  'Suntec City',
  'River Valley Primary School',
  'Zion Riverside Food Centre',
  'Clementi Town Secondary School',
  'NEWest',
  'Jurong Port'],
 ['Jurong Port',
  'Jurong Point',
  'Jurong West Primary School',
  'Keat Hong Food Centre & Market',
  'Fuchun Secondary School',
  'Kampung Admiralty',
  'Admiralty Primary School',
  'Sun Plaza',
  'Sembawang Hills Food Centre',
  'Adam Rd Food Centre',
  'Bukit Timah Plaza',
  'Jurong Port']]

In [32]:
import requests
from django.http import JsonResponse

def get_route(request):
    # Example coordinates: [(lon, lat), …]
    coords = [
        (103.851959, 1.290270),   # Marina Bay
        (103.819839, 1.352083),   # Ang Mo Kio
        (103.819836, 1.300083),   # Bugis
    ]

    coords_str = ";".join(f"{lon},{lat}" for lon, lat in coords)

    url = f"http://localhost:5000/route/v1/driving/{coords_str}?overview=full&geometries=geojson"

    resp = requests.get(url)
    data = resp.json()

    if "routes" not in data:
        return JsonResponse({"error": "No route found", "details": data})

    # Return the GeoJSON geometry
    route_geometry = data["routes"][0]["geometry"]

    return JsonResponse({"geometry": route_geometry, "coordinates": coords})


In [35]:
import requests
import folium

# List of waypoints (lat, lon)
waypoints = [
    (1.3521, 103.8198),  # Singapore
    (1.2903, 103.8519),  # Marina Bay
    (1.2831, 103.8603)   # Gardens by the Bay
]

# Build OSRM coords string: lon,lat;lon,lat;…
coords_str = ';'.join(f"{lon},{lat}" for lat, lon in waypoints)

# Query OSRM public API
url = f"http://router.project-osrm.org/route/v1/driving/{coords_str}"
params = {
    "overview": "full",
    "geometries": "geojson"
}
resp = requests.get(url, params=params)
resp.raise_for_status()
data = resp.json()

route_coords = data['routes'][0]['geometry']['coordinates']  # list of [lon,lat]
distance_km = data['routes'][0]['distance'] / 1000
duration_min = data['routes'][0]['duration'] / 60

print(f"Route distance: {distance_km:.2f} km")
print(f"Route duration: {duration_min:.1f} min")


Route distance: 13.69 km
Route duration: 23.3 min


In [47]:
# Swap lon,lat → lat,lon for Folium
route_latlng = [(lat, lon) for lon, lat in route_coords]

# Center map on first waypoint
m = folium.Map(location=waypoints[0], zoom_start=13)

# Add waypoints as markers
for idx, (lat, lon) in enumerate(waypoints, 1):
    folium.Marker(location=(lat, lon), popup=f"Point {idx}").add_to(m)

# Add route as polyline
folium.PolyLine(
    locations=route_latlng1,
    color="blue",
    weight=5,
    opacity=0.8,
    popup=f"Route: {distance_km:.2f} km, {duration_min:.1f} min"
).add_to(m)

# Save to HTML
m.save("osrm_route_map.html")
print("Map saved as osrm_route_map.html")


Map saved as osrm_route_map.html


In [45]:
route_latlng1 = [[103.800854, 1.440875], [103.800503, 1.44072], [103.800269, 1.440614], [103.80013, 1.440554], [103.800023, 1.440503], [103.799956, 1.440377], [103.799941, 1.440311], [103.799961, 1.440246], [103.80001, 1.440152], [103.800054, 1.440055], [103.800216, 1.439742], [103.800264, 1.439658], [103.800352, 1.439457], [103.800396, 1.439354], [103.800454, 1.439223], [103.800538, 1.439157], [103.800626, 1.439115], [103.800795, 1.439128], [103.801048, 1.439238], [103.801149, 1.439254], [103.801237, 1.439242], [103.801321, 1.439233], [103.80139, 1.439208], [103.801451, 1.439168], [103.80154, 1.439099], [103.801618, 1.438968], [103.801718, 1.438721], [103.801847, 1.438436], [103.801907, 1.438307], [103.801936, 1.43824], [103.802113, 1.437825], [103.802141, 1.437721], [103.802143, 1.437561], [103.802134, 1.437341], [103.802111, 1.437156], [103.801992, 1.436947], [103.801826, 1.436726], [103.801579, 1.436403], [103.80125, 1.435991], [103.801516, 1.435767], [103.801701, 1.435612], [103.801942, 1.435418], [103.802198, 1.43516], [103.80231, 1.435102], [103.802367, 1.435056], [103.802245, 1.434919], [103.802067, 1.434712], [103.801907, 1.434523], [103.801813, 1.434375], [103.801735, 1.434202], [103.801681, 1.433987], [103.801662, 1.433869], [103.801642, 1.433552], [103.801635, 1.433467], [103.801749, 1.4333], [103.801834, 1.433238], [103.80199, 1.43316], [103.802102, 1.433133], [103.802354, 1.433058], [103.802851, 1.432848], [103.803079, 1.432726], [103.803263, 1.432596], [103.803434, 1.432436], [103.803472, 1.43239], [103.803596, 1.432265], [103.803529, 1.432188], [103.80348, 1.432132], [103.803455, 1.432103], [103.802892, 1.431455], [103.801742, 1.430131], [103.800839, 1.429079], [103.799535, 1.427592], [103.799301, 1.427316], [103.799043, 1.42702], [103.798935, 1.426897], [103.798885, 1.426839], [103.798814, 1.426761], [103.798778, 1.42672], [103.798618, 1.426533], [103.798279, 1.426137], [103.798246, 1.426098], [103.796776, 1.424384], [103.796274, 1.423814], [103.796254, 1.423792], [103.796035, 1.423539], [103.795828, 1.423137], [103.795796, 1.422998], [103.795786, 1.422867], [103.795804, 1.422736], [103.795848, 1.422596], [103.795938, 1.42248], [103.796041, 1.422366], [103.796153, 1.422268], [103.796678, 1.421884], [103.797128, 1.42159], [103.797269, 1.42149], [103.797899, 1.421045], [103.79919, 1.420228], [103.799815, 1.419822], [103.800212, 1.419532], [103.800467, 1.419336], [103.800822, 1.419046], [103.802397, 1.417715], [103.803726, 1.416609], [103.804324, 1.416061], [103.804763, 1.415682], [103.805207, 1.41519], [103.805334, 1.415028], [103.805426, 1.414893], [103.805538, 1.414703], [103.8057, 1.414449], [103.806018, 1.413923], [103.806371, 1.413162], [103.806665, 1.412382], [103.806872, 1.411849], [103.807204, 1.410993], [103.807748, 1.409535], [103.807903, 1.409135], [103.808139, 1.408531], [103.808638, 1.407342], [103.808814, 1.407017], [103.809901, 1.405354], [103.810393, 1.404775], [103.810845, 1.4042], [103.811458, 1.403331], [103.811999, 1.402447], [103.812122, 1.40224], [103.812328, 1.401851], [103.81251, 1.401525], [103.812996, 1.40053], [103.813293, 1.399796], [103.813661, 1.398729], [103.814003, 1.397753], [103.814317, 1.3971], [103.814694, 1.396538], [103.814986, 1.396212], [103.815097, 1.3961], [103.815924, 1.395489], [103.815994, 1.395489], [103.816035, 1.395483], [103.816054, 1.395477], [103.816211, 1.395414], [103.816385, 1.39533], [103.816674, 1.395227], [103.817384, 1.395023], [103.817835, 1.394917], [103.81817, 1.39484], [103.818302, 1.394775], [103.818409, 1.394713], [103.818509, 1.394621], [103.81872, 1.394313], [103.818773, 1.394236], [103.81885, 1.394124], [103.818939, 1.393964], [103.81891, 1.393747], [103.818892, 1.393622], [103.818799, 1.393183], [103.818641, 1.392472], [103.818563, 1.392149], [103.818471, 1.391768], [103.81834, 1.391292], [103.81823, 1.39075], [103.818131, 1.390069], [103.818126, 1.389573], [103.818212, 1.389095], [103.818416, 1.388619], [103.818678, 1.388232], [103.818971, 1.387927], [103.819206, 1.387729], [103.819529, 1.387525], [103.819961, 1.387256], [103.820691, 1.3868], [103.821166, 1.386484], [103.821209, 1.386448], [103.821337, 1.386365], [103.821441, 1.386297], [103.823772, 1.384869], [103.824326, 1.38451], [103.824639, 1.384302], [103.824678, 1.384276], [103.825039, 1.383975], [103.825296, 1.383721], [103.825515, 1.383438], [103.825916, 1.382725], [103.826098, 1.382372], [103.826151, 1.382268], [103.826165, 1.382242], [103.826199, 1.382176], [103.826233, 1.38211], [103.826247, 1.382082], [103.826289, 1.382001], [103.826578, 1.381459], [103.826675, 1.381279], [103.827156, 1.380383], [103.827249, 1.380219], [103.827291, 1.380138], [103.827321, 1.380092], [103.827817, 1.379091], [103.828003, 1.378695], [103.828092, 1.378493], [103.828248, 1.378007], [103.82828, 1.377906], [103.828315, 1.377795], [103.828326, 1.377748], [103.828354, 1.377631], [103.828393, 1.377393], [103.82842, 1.377176], [103.828444, 1.376934], [103.828452, 1.376819], [103.82848, 1.376493], [103.828485, 1.376442], [103.828489, 1.376373], [103.828503, 1.376111], [103.828514, 1.375892], [103.828528, 1.375657], [103.828538, 1.375499], [103.828556, 1.375316], [103.828584, 1.374698], [103.828604, 1.374137], [103.828626, 1.373787], [103.828649, 1.373292], [103.828652, 1.373238], [103.828654, 1.373193], [103.828664, 1.373059], [103.828671, 1.372944], [103.828673, 1.372911], [103.828681, 1.372699], [103.828682, 1.372671], [103.828693, 1.372331], [103.8287, 1.372131], [103.828696, 1.37162], [103.828694, 1.371548], [103.828693, 1.371461], [103.828638, 1.370363], [103.82857, 1.369694], [103.828524, 1.369248], [103.828516, 1.368955], [103.828508, 1.368691], [103.828504, 1.368625], [103.828493, 1.368437], [103.828476, 1.368231], [103.828474, 1.368209], [103.82846, 1.368038], [103.82846, 1.367922], [103.828456, 1.367881], [103.828445, 1.367782], [103.82841, 1.367467], [103.828396, 1.367321], [103.828392, 1.367282], [103.828378, 1.367128], [103.828374, 1.36709], [103.828331, 1.366517], [103.828303, 1.366135], [103.828282, 1.365817], [103.828241, 1.365195], [103.828204, 1.36473], [103.828195, 1.364644], [103.828175, 1.364353], [103.828173, 1.363839], [103.828173, 1.363786], [103.828171, 1.36367], [103.828181, 1.363428], [103.828185, 1.363381], [103.828192, 1.363023], [103.828192, 1.362732], [103.828196, 1.362467], [103.828207, 1.362201], [103.828215, 1.362064], [103.828231, 1.361922], [103.828273, 1.361683], [103.828357, 1.361408], [103.828603, 1.360769], [103.828751, 1.360322], [103.828848, 1.359919], [103.828857, 1.359852], [103.828873, 1.359712], [103.828876, 1.359478], [103.828876, 1.359441], [103.828871, 1.35938], [103.828868, 1.359326], [103.82884, 1.359052], [103.828834, 1.358978], [103.828814, 1.358625], [103.828792, 1.358218], [103.828802, 1.358011], [103.828804, 1.358004], [103.828852, 1.357794], [103.82893, 1.357595], [103.828948, 1.357545], [103.828959, 1.357517], [103.829022, 1.357376], [103.829317, 1.357014], [103.83004, 1.356282], [103.830365, 1.355969], [103.830549, 1.355814], [103.830723, 1.355685], [103.830829, 1.355619], [103.83089, 1.355587], [103.831024, 1.355515], [103.831119, 1.355464], [103.831258, 1.355389], [103.83148, 1.355287], [103.831622, 1.355217], [103.831846, 1.355107], [103.831903, 1.355081], [103.832131, 1.354993], [103.832485, 1.35482], [103.832593, 1.354767], [103.832789, 1.354671], [103.832952, 1.354575], [103.833047, 1.354519], [103.833194, 1.35443], [103.833397, 1.354297], [103.833624, 1.354103], [103.833677, 1.354054], [103.833799, 1.353947], [103.833848, 1.353905], [103.833906, 1.353846], [103.833981, 1.353771], [103.834054, 1.353696], [103.834121, 1.353628], [103.834257, 1.353482], [103.834525, 1.353231], [103.834577, 1.353184], [103.83467, 1.3531], [103.834773, 1.353011], [103.835239, 1.352561], [103.835274, 1.352513], [103.835402, 1.352355], [103.835452, 1.352283], [103.835551, 1.352115], [103.835566, 1.35209], [103.835637, 1.351934], [103.835758, 1.351627], [103.835779, 1.351568], [103.835803, 1.35149], [103.835941, 1.351042], [103.836092, 1.350659], [103.836143, 1.350587], [103.836954, 1.349599], [103.837498, 1.348867], [103.837598, 1.348706], [103.837639, 1.34864], [103.83765, 1.348623], [103.837653, 1.348618], [103.837763, 1.348443], [103.837794, 1.348393], [103.837861, 1.348279], [103.837998, 1.34808], [103.838157, 1.34785], [103.83825, 1.347701], [103.838443, 1.347285], [103.838483, 1.347207], [103.838566, 1.346921], [103.838631, 1.346627], [103.83863, 1.346117], [103.838618, 1.346001], [103.838533, 1.344727], [103.838491, 1.344208], [103.838456, 1.343863], [103.838438, 1.343715], [103.838392, 1.343453], [103.838274, 1.343165], [103.838155, 1.342986], [103.83808, 1.342883], [103.837905, 1.34272], [103.837698, 1.342573], [103.836491, 1.341859], [103.836283, 1.341749], [103.835641, 1.341419], [103.834671, 1.340831], [103.834425, 1.340687], [103.83408, 1.340534], [103.833806, 1.340457], [103.833624, 1.340405], [103.833216, 1.34035], [103.832876, 1.340323], [103.832514, 1.340338], [103.832216, 1.340374], [103.832138, 1.340381], [103.831803, 1.340414], [103.831143, 1.340498], [103.830863, 1.340524], [103.830658, 1.340537], [103.830462, 1.340537], [103.830045, 1.34051], [103.829647, 1.340455], [103.829247, 1.340386], [103.828889, 1.340306], [103.828539, 1.340204], [103.82778, 1.339934], [103.827327, 1.339762], [103.826838, 1.339542], [103.82642, 1.339328], [103.825969, 1.339065], [103.825379, 1.338692], [103.823534, 1.337429], [103.82233, 1.336633], [103.821942, 1.33636], [103.821723, 1.336216], [103.820997, 1.335726], [103.820698, 1.33551], [103.820289, 1.335265], [103.820096, 1.335162], [103.819881, 1.335052], [103.819678, 1.334955], [103.818961, 1.334597], [103.818768, 1.334504], [103.818408, 1.334319], [103.818213, 1.334207], [103.817969, 1.334052], [103.817756, 1.333893], [103.817541, 1.333719], [103.817354, 1.33355], [103.817203, 1.333393], [103.817031, 1.333189], [103.816845, 1.332935], [103.816398, 1.332298], [103.816323, 1.332182], [103.816212, 1.332013], [103.816154, 1.33193], [103.815894, 1.331475], [103.815872, 1.331429], [103.815703, 1.331076], [103.815617, 1.330915], [103.815407, 1.330577], [103.815209, 1.330253], [103.815129, 1.330135], [103.815048, 1.330018], [103.814941, 1.329858], [103.814916, 1.329821], [103.814812, 1.329668], [103.814722, 1.329533], [103.814403, 1.32906], [103.81427, 1.328789], [103.814217, 1.328663], [103.814175, 1.328551], [103.814133, 1.328424], [103.814091, 1.328275], [103.814047, 1.328082], [103.814019, 1.327954], [103.813983, 1.327704], [103.81396, 1.32745], [103.813924, 1.326885], [103.813853, 1.326144], [103.813762, 1.325204], [103.813715, 1.324881], [103.813646, 1.324555], [103.813584, 1.324298], [103.813552, 1.324132], [103.813366, 1.323233], [103.813191, 1.322227], [103.813144, 1.322046], [103.813064, 1.321821], [103.813002, 1.321693], [103.812903, 1.321538], [103.812719, 1.321312], [103.812273, 1.320805], [103.811971, 1.320479], [103.811773, 1.320281], [103.811544, 1.320078], [103.811411, 1.319977], [103.81118, 1.31984], [103.810712, 1.319587], [103.810107, 1.319308], [103.809942, 1.319233], [103.809719, 1.319125], [103.809519, 1.319], [103.80944, 1.318958], [103.809386, 1.318921], [103.809199, 1.318778], [103.808365, 1.318035], [103.807516, 1.31725], [103.807454, 1.317194], [103.807394, 1.317139], [103.80732, 1.317068], [103.807258, 1.317012], [103.807202, 1.316961], [103.80688, 1.316654], [103.806736, 1.316513], [103.806359, 1.316192], [103.80619, 1.316034], [103.805891, 1.315751], [103.80582, 1.315682], [103.805441, 1.315315], [103.805227, 1.315088], [103.805202, 1.315064], [103.805179, 1.315035], [103.805157, 1.315008], [103.805051, 1.314882], [103.804854, 1.314593], [103.804721, 1.31429], [103.804603, 1.313997], [103.804556, 1.313858], [103.804509, 1.31373], [103.804407, 1.313454], [103.804355, 1.313306], [103.804329, 1.313221], [103.804269, 1.313054], [103.804216, 1.312896], [103.804089, 1.312548], [103.803759, 1.311342], [103.803523, 1.310436], [103.803326, 1.309837], [103.80303, 1.309071], [103.802985, 1.308954], [103.802768, 1.308412], [103.802348, 1.307261], [103.802226, 1.306931], [103.801971, 1.306204], [103.801829, 1.305669], [103.801799, 1.305499], [103.801777, 1.305378], [103.801771, 1.305344], [103.801732, 1.305122], [103.80172, 1.304996], [103.801703, 1.30472], [103.801684, 1.30439], [103.801639, 1.30386], [103.801624, 1.30369], [103.801601, 1.303479], [103.801544, 1.303184], [103.801396, 1.302408], [103.801373, 1.30229], [103.801246, 1.301643], [103.801014, 1.300527], [103.800839, 1.299714], [103.800686, 1.299013], [103.800608, 1.298687], [103.800567, 1.298519], [103.800541, 1.298401], [103.800489, 1.29817], [103.800385, 1.297789], [103.800295, 1.297501], [103.800198, 1.297204], [103.800084, 1.296882], [103.79966, 1.295936], [103.799591, 1.295782], [103.799483, 1.295489], [103.799428, 1.295334], [103.799405, 1.295238], [103.799357, 1.295038], [103.799302, 1.294807], [103.799104, 1.29387], [103.799086, 1.293815], [103.799064, 1.293735], [103.798998, 1.293541], [103.798939, 1.293383], [103.798896, 1.293283], [103.798807, 1.293106], [103.79871, 1.292886], [103.798509, 1.292579], [103.79828, 1.292302], [103.797937, 1.291973], [103.797654, 1.291784], [103.797353, 1.291604], [103.79697, 1.29142], [103.796495, 1.291229], [103.796116, 1.291067], [103.79603, 1.29103], [103.795842, 1.290947], [103.795716, 1.290897], [103.795317, 1.290725], [103.795247, 1.290696], [103.795117, 1.290641], [103.795092, 1.290631], [103.794924, 1.290554], [103.794831, 1.290519], [103.794629, 1.290436], [103.794239, 1.290227], [103.793817, 1.289946], [103.793672, 1.289833], [103.793555, 1.289706], [103.793543, 1.289692], [103.793412, 1.289541], [103.793256, 1.289381], [103.793166, 1.289286], [103.793134, 1.289252], [103.792736, 1.288833], [103.792713, 1.288808], [103.792663, 1.288846], [103.792615, 1.288885], [103.792588, 1.288907], [103.792501, 1.288977], [103.792292, 1.289151], [103.792009, 1.289394], [103.791771, 1.289598], [103.791636, 1.289714], [103.791452, 1.289871], [103.791095, 1.290177], [103.790443, 1.290633], [103.789756, 1.291152], [103.78871, 1.291827], [103.788507, 1.291947], [103.787773, 1.29246], [103.787654, 1.29256], [103.787185, 1.292987], [103.786515, 1.293521], [103.786309, 1.293732], [103.786094, 1.293988], [103.78575, 1.294425], [103.785684, 1.294514], [103.785578, 1.294635], [103.785474, 1.294763], [103.785242, 1.29502], [103.784644, 1.295725], [103.784384, 1.296024], [103.784231, 1.296199], [103.784179, 1.296259], [103.783862, 1.296605], [103.783394, 1.297042], [103.783111, 1.297269], [103.782811, 1.297482], [103.782304, 1.297773], [103.781775, 1.298086], [103.781152, 1.298431], [103.780483, 1.298811], [103.780076, 1.299038], [103.780026, 1.299065], [103.778859, 1.299784], [103.778791, 1.299832], [103.778592, 1.299955], [103.777848, 1.300414], [103.777446, 1.300628], [103.776743, 1.301062], [103.776478, 1.301223], [103.774543, 1.302329], [103.774462, 1.302375], [103.774401, 1.302415], [103.774006, 1.302645], [103.77338, 1.302986], [103.773137, 1.303142], [103.772401, 1.303514], [103.771818, 1.303773], [103.771396, 1.303965], [103.77095, 1.304164], [103.77019, 1.304553], [103.769541, 1.304937], [103.768953, 1.305351], [103.7683, 1.305806], [103.76791, 1.306121], [103.767695, 1.306303], [103.767162, 1.30677], [103.766728, 1.30723], [103.766649, 1.307351], [103.766506, 1.30755], [103.766011, 1.30823], [103.765784, 1.308541], [103.765541, 1.308876], [103.764981, 1.309576], [103.764862, 1.309716], [103.764447, 1.310181], [103.764332, 1.310316], [103.764154, 1.310525], [103.764096, 1.310583], [103.763983, 1.310697], [103.763761, 1.310919], [103.763575, 1.311107], [103.763532, 1.311149], [103.763354, 1.311323], [103.763304, 1.311371], [103.762754, 1.311905], [103.762379, 1.312314], [103.761446, 1.31322], [103.761307, 1.31336], [103.760528, 1.314138], [103.760246, 1.31442], [103.759958, 1.314708], [103.758469, 1.316364], [103.75753, 1.31747], [103.757341, 1.317692], [103.756324, 1.318936], [103.75601, 1.319321], [103.755411, 1.319996], [103.755286, 1.320136], [103.755191, 1.320243], [103.754911, 1.320542], [103.754875, 1.320569], [103.754583, 1.320789], [103.754319, 1.320931], [103.754055, 1.321073], [103.754025, 1.321083], [103.752889, 1.321456], [103.752313, 1.321628], [103.752187, 1.321668], [103.751524, 1.321908], [103.750834, 1.322118], [103.750607, 1.322182], [103.749911, 1.322363], [103.749775, 1.322394], [103.749587, 1.322449], [103.749494, 1.322476], [103.749018, 1.322591], [103.748714, 1.32265], [103.747227, 1.322972], [103.746047, 1.323207], [103.745419, 1.323334], [103.745306, 1.323357], [103.744817, 1.323459], [103.744442, 1.323534], [103.743775, 1.323718], [103.743551, 1.323814], [103.743279, 1.32394], [103.74281, 1.324195], [103.741644, 1.324955], [103.741051, 1.325341], [103.73985, 1.326174], [103.739749, 1.326258], [103.739594, 1.326367], [103.739293, 1.326569], [103.7387, 1.326864], [103.738188, 1.327052], [103.73736, 1.327249], [103.736747, 1.327291], [103.736136, 1.327256], [103.735377, 1.327108], [103.733957, 1.32674], [103.733324, 1.326593], [103.732998, 1.326517], [103.729697, 1.325735], [103.729434, 1.325668], [103.729237, 1.325623], [103.728202, 1.32539], [103.728055, 1.325293], [103.727734, 1.325193], [103.726476, 1.324872], [103.726199, 1.324824], [103.725911, 1.324764], [103.725456, 1.324663], [103.725015, 1.324561], [103.723044, 1.32409], [103.722753, 1.324022], [103.721889, 1.323831], [103.721475, 1.323743], [103.721439, 1.323735], [103.72122, 1.32369], [103.721124, 1.323671], [103.720889, 1.323615], [103.72052, 1.32352], [103.720168, 1.323389], [103.720056, 1.32324], [103.719944, 1.32309], [103.719911, 1.323012], [103.719893, 1.322928], [103.719867, 1.322746], [103.720057, 1.321809], [103.720083, 1.321668], [103.720105, 1.321551], [103.720131, 1.321402], [103.720195, 1.321108], [103.72027, 1.32076], [103.720318, 1.320525], [103.720331, 1.320463], [103.720347, 1.32037], [103.720356, 1.320299], [103.720371, 1.320133], [103.720399, 1.319788], [103.7204, 1.319734], [103.720407, 1.319269], [103.720399, 1.319029], [103.72039, 1.318949], [103.72037, 1.318773], [103.720315, 1.318292], [103.720281, 1.318001], [103.72026, 1.31792], [103.720144, 1.317302], [103.720006, 1.316595], [103.719992, 1.316524], [103.719976, 1.316431], [103.71995, 1.316289], [103.71989, 1.31593], [103.719874, 1.31584], [103.719828, 1.315568], [103.719779, 1.315281], [103.71977, 1.315229], [103.719694, 1.314791], [103.719625, 1.314422], [103.719612, 1.314352], [103.719587, 1.314215], [103.719556, 1.314078], [103.719481, 1.31377], [103.719223, 1.312677], [103.719089, 1.31211], [103.719068, 1.312022], [103.718957, 1.311553], [103.718902, 1.311237], [103.718864, 1.311016], [103.718856, 1.310968], [103.718775, 1.310802], [103.718587, 1.310505], [103.718304, 1.310086], [103.718176, 1.309953], [103.717816, 1.309539], [103.7173, 1.30898], [103.716908, 1.308536], [103.717699, 1.307869], [103.717889, 1.307704], [103.718445, 1.307232], [103.720541, 1.305455], [103.720807, 1.305235], [103.721003, 1.305474], [103.7205, 1.30589], [103.721361, 1.306935], [103.721672, 1.307332], [103.72176, 1.307359], [103.722259, 1.30701], [103.7222, 1.306938], [103.722041, 1.306744], [103.721179, 1.305689], [103.721003, 1.305474], [103.720807, 1.305235], [103.720541, 1.305455], [103.718445, 1.307232], [103.717889, 1.307704], [103.717699, 1.307869], [103.716908, 1.308536], [103.716813, 1.308612], [103.717557, 1.309501], [103.717666, 1.309666], [103.71783, 1.310047], [103.717887, 1.310172], [103.718008, 1.310369], [103.718143, 1.310496], [103.718346, 1.310674], [103.718494, 1.31078], [103.718601, 1.310895], [103.718703, 1.311024], [103.718821, 1.311264], [103.718879, 1.311572], [103.719002, 1.312129], [103.719031, 1.31226], [103.719128, 1.3127], [103.719222, 1.313088], [103.71929, 1.313398], [103.719372, 1.313813], [103.719436, 1.314102], [103.719466, 1.314237], [103.719495, 1.314375], [103.719508, 1.314443], [103.719579, 1.314812], [103.719673, 1.315307], [103.719742, 1.315673], [103.719778, 1.315862], [103.719794, 1.315948], [103.719862, 1.316307], [103.719889, 1.316448], [103.719907, 1.316541], [103.719921, 1.316611], [103.720063, 1.317319], [103.720186, 1.317934], [103.720204, 1.31802], [103.720232, 1.318304], [103.72028, 1.318781], [103.720307, 1.319042], [103.720305, 1.319269], [103.720294, 1.319731], [103.720279, 1.320054], [103.720268, 1.320288], [103.720262, 1.320351], [103.72025, 1.320454], [103.720237, 1.320514], [103.720202, 1.320671], [103.720046, 1.321374], [103.720043, 1.321388], [103.720015, 1.321507], [103.71999, 1.321635], [103.719961, 1.321785], [103.719792, 1.32255], [103.719713, 1.322854], [103.719658, 1.323129], [103.71965, 1.323173], [103.719616, 1.323325], [103.719486, 1.32387], [103.719618, 1.323901], [103.719695, 1.323905], [103.719902, 1.323915], [103.720016, 1.323906], [103.720132, 1.3239], [103.720435, 1.323924], [103.72113, 1.324073], [103.721363, 1.324123], [103.721874, 1.324234], [103.722717, 1.324459], [103.722944, 1.324524], [103.723537, 1.324689], [103.723834, 1.324706], [103.724404, 1.324831], [103.724867, 1.324934], [103.724944, 1.324951], [103.725708, 1.325131], [103.726226, 1.325247], [103.727391, 1.325517], [103.727445, 1.32553], [103.72785, 1.325635], [103.728292, 1.325735], [103.728608, 1.325783], [103.729449, 1.325913], [103.732963, 1.326687], [103.733288, 1.326762], [103.733903, 1.326903], [103.735359, 1.32727], [103.736128, 1.327418], [103.736753, 1.327447], [103.737345, 1.327413], [103.738252, 1.327213], [103.738476, 1.327142], [103.738764, 1.327028], [103.739019, 1.326902], [103.739278, 1.326773], [103.739371, 1.326727], [103.739529, 1.32662], [103.739671, 1.326517], [103.739728, 1.326477], [103.740312, 1.326083], [103.74211, 1.324845], [103.742948, 1.324282], [103.743024, 1.324242], [103.743332, 1.324091], [103.743594, 1.323961], [103.743842, 1.323862], [103.744485, 1.323677], [103.745345, 1.323499], [103.745449, 1.323478], [103.746058, 1.323348], [103.747188, 1.323097], [103.748713, 1.32276], [103.749025, 1.322695], [103.750371, 1.322417], [103.750886, 1.322267], [103.751589, 1.322069], [103.752228, 1.321852], [103.753074, 1.321577], [103.753574, 1.321407], [103.753799, 1.321331], [103.754155, 1.321209], [103.754433, 1.32105], [103.754711, 1.320891], [103.755014, 1.320627], [103.755342, 1.320325], [103.755437, 1.32021], [103.755994, 1.319535], [103.757027, 1.31835], [103.757102, 1.31827], [103.757468, 1.317841], [103.757537, 1.317762], [103.757594, 1.317696], [103.757667, 1.31761], [103.758064, 1.31715], [103.758317, 1.316851], [103.758693, 1.316426], [103.759003, 1.316075], [103.759082, 1.315986], [103.76007, 1.314819], [103.76027, 1.314602], [103.760577, 1.314297], [103.761419, 1.313461], [103.761571, 1.31331], [103.762201, 1.312686], [103.762968, 1.311913], [103.763099, 1.31178], [103.763438, 1.311439], [103.763489, 1.311387], [103.763522, 1.311353], [103.763628, 1.311247], [103.764243, 1.310625], [103.764477, 1.31039], [103.764542, 1.310324], [103.764962, 1.309872], [103.765045, 1.309762], [103.765688, 1.308896], [103.765865, 1.308654], [103.766069, 1.308376], [103.766267, 1.308104], [103.766457, 1.307843], [103.766763, 1.307436], [103.767084, 1.307128], [103.767283, 1.306919], [103.767664, 1.306581], [103.768088, 1.306246], [103.768623, 1.305812], [103.768755, 1.305701], [103.76902, 1.30549], [103.769621, 1.305083], [103.771006, 1.304337], [103.771416, 1.304155], [103.771977, 1.303916], [103.77334, 1.30325], [103.773503, 1.303169], [103.774022, 1.30285], [103.775188, 1.302176], [103.776593, 1.301362], [103.777965, 1.300558], [103.778159, 1.300436], [103.778264, 1.30037], [103.778664, 1.300121], [103.778895, 1.299976], [103.779384, 1.299672], [103.779521, 1.299589], [103.780588, 1.298945], [103.781021, 1.298705], [103.781226, 1.298592], [103.781852, 1.298248], [103.782395, 1.297939], [103.783111, 1.29752], [103.783504, 1.297224], [103.783573, 1.29716], [103.783661, 1.297079], [103.784022, 1.296743], [103.784384, 1.296317], [103.78453, 1.296145], [103.784788, 1.295842], [103.78575, 1.294758], [103.786158, 1.294232], [103.786496, 1.293818], [103.787227, 1.293081], [103.787808, 1.292611], [103.788448, 1.292214], [103.790368, 1.291009], [103.792214, 1.2898], [103.792682, 1.289443], [103.792835, 1.289317], [103.792873, 1.289286], [103.793167, 1.289045], [103.793262, 1.288967], [103.793433, 1.288826], [103.794299, 1.288246], [103.794661, 1.288021], [103.795367, 1.287623], [103.79559, 1.287497], [103.795939, 1.287301], [103.796504, 1.28693], [103.796795, 1.286732], [103.797019, 1.28659], [103.797356, 1.286367], [103.797971, 1.285944], [103.798255, 1.28575], [103.798394, 1.285648], [103.799145, 1.285104], [103.799713, 1.284745], [103.800227, 1.284499], [103.800764, 1.284318], [103.80131, 1.284208], [103.802068, 1.284119], [103.806036, 1.283884], [103.807955, 1.283769], [103.808244, 1.283752], [103.810145, 1.283639], [103.810908, 1.283566], [103.811687, 1.283453], [103.812417, 1.283281], [103.813144, 1.283052], [103.813795, 1.282802], [103.814557, 1.282432], [103.81499, 1.282186], [103.815279, 1.282022], [103.817047, 1.280912], [103.817446, 1.28067], [103.817657, 1.280534], [103.818168, 1.280203], [103.818503, 1.280001], [103.818824, 1.279797], [103.819296, 1.279524], [103.820344, 1.279151], [103.820514, 1.279103], [103.820657, 1.279068], [103.821065, 1.278992], [103.821261, 1.278962], [103.821552, 1.278945], [103.82227, 1.278941], [103.822547, 1.27894], [103.823258, 1.278882], [103.823336, 1.278835], [103.823415, 1.278803], [103.823547, 1.278762], [103.823868, 1.278637], [103.824157, 1.278523], [103.824486, 1.278371], [103.825024, 1.278076], [103.825681, 1.277707], [103.825977, 1.277578], [103.826381, 1.277477], [103.826714, 1.277451], [103.827059, 1.277492], [103.827342, 1.277558], [103.827601, 1.277653], [103.827881, 1.277811], [103.828206, 1.278077], [103.828386, 1.278267], [103.828552, 1.278482], [103.828713, 1.278717], [103.828957, 1.279068], [103.829194, 1.27939], [103.829466, 1.279712], [103.829751, 1.280023], [103.829913, 1.280196], [103.830135, 1.280408], [103.830357, 1.280602], [103.830705, 1.280918], [103.831162, 1.281244], [103.831338, 1.281355], [103.831683, 1.281573], [103.832105, 1.281821], [103.832559, 1.282046], [103.833242, 1.282312], [103.833393, 1.282369], [103.83345, 1.282421], [103.833486, 1.282437], [103.833503, 1.282447], [103.833896, 1.282647], [103.834335, 1.282838], [103.835045, 1.283091], [103.835271, 1.283175], [103.835443, 1.283239], [103.835558, 1.283354], [103.835573, 1.283412], [103.835555, 1.283535], [103.835394, 1.283803], [103.835332, 1.28388], [103.835265, 1.283931], [103.835218, 1.283954], [103.835094, 1.284015], [103.835024, 1.284043], [103.834968, 1.284059], [103.834799, 1.284117], [103.834831, 1.284272], [103.834856, 1.284362], [103.834885, 1.284401], [103.834932, 1.284429], [103.835158, 1.284506], [103.835184, 1.284448], [103.83526, 1.284303], [103.835311, 1.284208], [103.835362, 1.284113], [103.835431, 1.284008], [103.835488, 1.283937], [103.83558, 1.283811], [103.835676, 1.283681], [103.835776, 1.283556], [103.835877, 1.283421], [103.836028, 1.283261], [103.836149, 1.283154], [103.836204, 1.283108], [103.836459, 1.282922], [103.837067, 1.282555], [103.83748, 1.282277], [103.837615, 1.282193], [103.838017, 1.281941], [103.838226, 1.281811], [103.838357, 1.281709], [103.838416, 1.281663], [103.838537, 1.281578], [103.838771, 1.281379], [103.838822, 1.281326], [103.838939, 1.281203], [103.839082, 1.281043], [103.839166, 1.280946], [103.839223, 1.280859], [103.839387, 1.280543], [103.839443, 1.280424], [103.839557, 1.280336], [103.839671, 1.280287], [103.839711, 1.280273], [103.839745, 1.280266], [103.839794, 1.280265], [103.839866, 1.280277], [103.839934, 1.280306], [103.840058, 1.280406], [103.840126, 1.280501], [103.840248, 1.280657], [103.840278, 1.280698], [103.840389, 1.280833], [103.840455, 1.280914], [103.840679, 1.281163], [103.840995, 1.281522], [103.841201, 1.281776], [103.84124, 1.281821], [103.841337, 1.281938], [103.841484, 1.282115], [103.841536, 1.282179], [103.841573, 1.28222], [103.841635, 1.282291], [103.841922, 1.282618], [103.84196, 1.282662], [103.842194, 1.282957], [103.842363, 1.283164], [103.842501, 1.283337], [103.842529, 1.283371], [103.843116, 1.284079], [103.843197, 1.284176], [103.843265, 1.284258], [103.843378, 1.284395], [103.843446, 1.284477], [103.843649, 1.28472], [103.843685, 1.284731], [103.843736, 1.284736], [103.843792, 1.284729], [103.843844, 1.284705], [103.843879, 1.284678], [103.843678, 1.284436], [103.843617, 1.284363], [103.843317, 1.284013], [103.843263, 1.283948], [103.84323, 1.283908], [103.842949, 1.283578], [103.842985, 1.283554], [103.843218, 1.283417], [103.843275, 1.283383], [103.843896, 1.283016], [103.843913, 1.283006], [103.844105, 1.282902], [103.844125, 1.282892], [103.844128, 1.28289], [103.844154, 1.282874], [103.844346, 1.282765], [103.844428, 1.282718], [103.844989, 1.282406], [103.845195, 1.282278], [103.845296, 1.282217], [103.845133, 1.281927], [103.845062, 1.2818], [103.845012, 1.281712], [103.844882, 1.281491], [103.844826, 1.281338], [103.844751, 1.281225], [103.844468, 1.280798], [103.844419, 1.280724], [103.844356, 1.280632], [103.844208, 1.280433], [103.844195, 1.2804], [103.844231, 1.280372], [103.844264, 1.280335], [103.844359, 1.280232], [103.844626, 1.279901], [103.844728, 1.279791], [103.844749, 1.279768], [103.844787, 1.279724], [103.844896, 1.279593], [103.845145, 1.279248], [103.845252, 1.279096], [103.84531, 1.279019], [103.845324, 1.278937], [103.845568, 1.278625], [103.845739, 1.278401], [103.846171, 1.27786], [103.846208, 1.277815], [103.846256, 1.277796], [103.846386, 1.277684], [103.846591, 1.277485], [103.84673, 1.277473], [103.846781, 1.277465], [103.846824, 1.277465], [103.846915, 1.277497], [103.846952, 1.277519], [103.847061, 1.277607], [103.847074, 1.277634], [103.847196, 1.277841], [103.847275, 1.277963], [103.84757, 1.27845], [103.847738, 1.278716], [103.847887, 1.278946], [103.848068, 1.279225], [103.848094, 1.279266], [103.848118, 1.279305], [103.84858, 1.280064], [103.848757, 1.280361], [103.848858, 1.280538], [103.84903, 1.280838], [103.849052, 1.280878], [103.849074, 1.280914], [103.849219, 1.281159], [103.849324, 1.281335], [103.849477, 1.281593], [103.849524, 1.281672], [103.849548, 1.281709], [103.849748, 1.282019], [103.849874, 1.282208], [103.849892, 1.282234], [103.850046, 1.282452], [103.850166, 1.282572], [103.850271, 1.282635], [103.850371, 1.282676], [103.850425, 1.282693], [103.850787, 1.282645], [103.850911, 1.282675], [103.851216, 1.282666], [103.85141, 1.282674], [103.851527, 1.28267], [103.851588, 1.282662], [103.85175, 1.282622], [103.851813, 1.282635], [103.851928, 1.282677], [103.851983, 1.282706], [103.852017, 1.28273], [103.852046, 1.282749], [103.852085, 1.282773], [103.85219, 1.282837], [103.852292, 1.282944], [103.852394, 1.283068], [103.852471, 1.283194], [103.852557, 1.283393], [103.852679, 1.283765], [103.852782, 1.284027], [103.852826, 1.284139], [103.852912, 1.284378], [103.852954, 1.284497], [103.853272, 1.28516], [103.853394, 1.285436], [103.853472, 1.285638], [103.853531, 1.285791], [103.853625, 1.28607], [103.853659, 1.286198], [103.853693, 1.286327], [103.853711, 1.286394], [103.853763, 1.286605], [103.85377, 1.286646], [103.85378, 1.286701], [103.854266, 1.289261], [103.85435, 1.289643], [103.854448, 1.290127], [103.85453, 1.29044], [103.854552, 1.290518], [103.854763, 1.291147], [103.854866, 1.291451], [103.854903, 1.291536], [103.854925, 1.291587], [103.85508, 1.291882], [103.855294, 1.292289], [103.855412, 1.292499], [103.85563, 1.292851], [103.855823, 1.293172], [103.855878, 1.293263], [103.856059, 1.293567], [103.85609, 1.293611], [103.856347, 1.294008], [103.856563, 1.294365], [103.856582, 1.294397], [103.856782, 1.294727], [103.856806, 1.294767], [103.857018, 1.295114], [103.857058, 1.295178], [103.857098, 1.295244], [103.857292, 1.295547], [103.857357, 1.295647], [103.857456, 1.295782], [103.857554, 1.295901], [103.857682, 1.296054], [103.857877, 1.296299], [103.858391, 1.296864], [103.858485, 1.296957], [103.859311, 1.297781], [103.85951, 1.297966], [103.859714, 1.29815], [103.859925, 1.298328], [103.859972, 1.298367], [103.860513, 1.298787], [103.861562, 1.299543], [103.862079, 1.299936], [103.862161, 1.299992], [103.862793, 1.300473], [103.863154, 1.300744], [103.86318, 1.300763], [103.863311, 1.300861], [103.863497, 1.300996], [103.864433, 1.301673], [103.865222, 1.302239], [103.86639, 1.302981], [103.8694, 1.30496], [103.869624, 1.305105], [103.869844, 1.305247], [103.870022, 1.305371], [103.870215, 1.305504], [103.870649, 1.305775], [103.871036, 1.306031], [103.871185, 1.306137], [103.871489, 1.306327], [103.871602, 1.306461], [103.871812, 1.306586], [103.87224, 1.306784], [103.87261, 1.306899], [103.87343, 1.306973], [103.87356, 1.306989], [103.873855, 1.307022], [103.874105, 1.30705], [103.874426, 1.307088], [103.874582, 1.307106], [103.874668, 1.307116], [103.875048, 1.307205], [103.875142, 1.307239], [103.875377, 1.307327], [103.875637, 1.307449], [103.875872, 1.307607], [103.875938, 1.307685], [103.875982, 1.307819], [103.875991, 1.307966], [103.875972, 1.308278], [103.875777, 1.309115], [103.875756, 1.309222], [103.875526, 1.310197], [103.875174, 1.311825], [103.874826, 1.313792], [103.874804, 1.313914], [103.874703, 1.314407], [103.874504, 1.315496], [103.874478, 1.315923], [103.874473, 1.316547], [103.874469, 1.316656], [103.874491, 1.316923], [103.874538, 1.317236], [103.874596, 1.317514], [103.874649, 1.317703], [103.874819, 1.318457], [103.874869, 1.318659], [103.874958, 1.318939], [103.875026, 1.319081], [103.875102, 1.319192], [103.875212, 1.319366], [103.875304, 1.319475], [103.875412, 1.31958], [103.875607, 1.31974], [103.875853, 1.31987], [103.876075, 1.319962], [103.876314, 1.32004], [103.876537, 1.320097], [103.876774, 1.320138], [103.877143, 1.320189], [103.877445, 1.320198], [103.877772, 1.320191], [103.878236, 1.320173], [103.878852, 1.320165], [103.879093, 1.320178], [103.879269, 1.320194], [103.879399, 1.320213], [103.879534, 1.320242], [103.879677, 1.32029], [103.8798, 1.320337], [103.879866, 1.320362], [103.88014, 1.320393], [103.881334, 1.320881], [103.882004, 1.321157], [103.883376, 1.321721], [103.884206, 1.322063], [103.884578, 1.322186], [103.884794, 1.322237], [103.884912, 1.32226], [103.884985, 1.322274], [103.885012, 1.322279], [103.885103, 1.322292], [103.885352, 1.322325], [103.885618, 1.322342], [103.885926, 1.322355], [103.88618, 1.322341], [103.886375, 1.32234], [103.887449, 1.322368], [103.887919, 1.322418], [103.88851, 1.322489], [103.889123, 1.322587], [103.891082, 1.323014], [103.892464, 1.323345], [103.893413, 1.323571], [103.894849, 1.323885], [103.895242, 1.323972], [103.895621, 1.324063], [103.896628, 1.324253], [103.898603, 1.324643], [103.900924, 1.325097], [103.902515, 1.325397], [103.902848, 1.325458], [103.902874, 1.325463], [103.902964, 1.325479], [103.903202, 1.325523], [103.903799, 1.325661], [103.904717, 1.325918], [103.907099, 1.326632], [103.907392, 1.32672], [103.908453, 1.32705], [103.908667, 1.327129], [103.908858, 1.327205], [103.909247, 1.327397], [103.909464, 1.327516], [103.910744, 1.32826], [103.91133, 1.328599], [103.911846, 1.32888], [103.912558, 1.329227], [103.913412, 1.329627], [103.915018, 1.330393], [103.917201, 1.331412], [103.91774, 1.331674], [103.917921, 1.331764], [103.918511, 1.332046], [103.919267, 1.332383], [103.919305, 1.332402], [103.919586, 1.332522], [103.920228, 1.332831], [103.921048, 1.333211], [103.921863, 1.333589], [103.923742, 1.334444], [103.924022, 1.334566], [103.924173, 1.334627], [103.924322, 1.334684], [103.924487, 1.334742], [103.925023, 1.334917], [103.925242, 1.334986], [103.925607, 1.335091], [103.925874, 1.335161], [103.926131, 1.335221], [103.926449, 1.335286], [103.926744, 1.335341], [103.92721, 1.335424], [103.927764, 1.335514], [103.928219, 1.335559], [103.928321, 1.335567], [103.928599, 1.335584], [103.928862, 1.3356], [103.929163, 1.335611], [103.929388, 1.335615], [103.930085, 1.335603], [103.931989, 1.335493], [103.933018, 1.335418], [103.934121, 1.335344], [103.935391, 1.33526], [103.936894, 1.33516], [103.937317, 1.335136], [103.937774, 1.33512], [103.938135, 1.335122], [103.938345, 1.335132], [103.93846, 1.335144], [103.938574, 1.335159], [103.938689, 1.335179], [103.938803, 1.335205], [103.93892, 1.335239], [103.939035, 1.335277], [103.939189, 1.335336], [103.939336, 1.335408], [103.93945, 1.335477], [103.940026, 1.335895], [103.940699, 1.336408], [103.942001, 1.337511], [103.943922, 1.33914], [103.945001, 1.340057], [103.9454, 1.340372], [103.945588, 1.340529], [103.946626, 1.341392], [103.947047, 1.341726], [103.947271, 1.341911], [103.947729, 1.342305], [103.948388, 1.342881], [103.948841, 1.343259], [103.949753, 1.344042], [103.950156, 1.34439], [103.951878, 1.345782], [103.953046, 1.34678], [103.954745, 1.348238], [103.955229, 1.348694], [103.955759, 1.349086], [103.955851, 1.34916], [103.956334, 1.349516], [103.95659, 1.349678], [103.956879, 1.349817], [103.957242, 1.350002], [103.957659, 1.350184], [103.958179, 1.350428], [103.958383, 1.350524], [103.959026, 1.350701], [103.959454, 1.350756], [103.959914, 1.350799], [103.960306, 1.350829], [103.960646, 1.350833], [103.96108, 1.350771], [103.961232, 1.350774], [103.961311, 1.350775], [103.961412, 1.350765], [103.961585, 1.35075], [103.961787, 1.350722], [103.962094, 1.350699], [103.962305, 1.350709], [103.962559, 1.350747], [103.962812, 1.350863], [103.963026, 1.351015], [103.963242, 1.351216], [103.963423, 1.351512], [103.963538, 1.351815], [103.963741, 1.352397], [103.963802, 1.352489], [103.963844, 1.35254], [103.963873, 1.352582], [103.963964, 1.352631], [103.964386, 1.352769], [103.964657, 1.352848], [103.964794, 1.352887], [103.964901, 1.352919], [103.965431, 1.353073], [103.965668, 1.35315], [103.966265, 1.353341], [103.966457, 1.353397], [103.966665, 1.353462], [103.966842, 1.353523], [103.967088, 1.353599], [103.967265, 1.353651], [103.967385, 1.35369], [103.967531, 1.353749], [103.967835, 1.353859], [103.96799, 1.353926], [103.968183, 1.354061], [103.968325, 1.354208], [103.968408, 1.354315], [103.968446, 1.354384], [103.968484, 1.354455], [103.96858, 1.354667], [103.968674, 1.354937], [103.96876, 1.355266], [103.969114, 1.356628], [103.969256, 1.357128], [103.969372, 1.357596], [103.969392, 1.357667], [103.969523, 1.358137], [103.969582, 1.358357], [103.969601, 1.358427], [103.969718, 1.358877], [103.969723, 1.358896], [103.969739, 1.358957], [103.96986, 1.358928], [103.969907, 1.358915], [103.969954, 1.358902], [103.969907, 1.358915], [103.96986, 1.358928], [103.969838, 1.358849], [103.96964, 1.358146], [103.969614, 1.358053], [103.969563, 1.357874], [103.969498, 1.357643], [103.969478, 1.357569], [103.969252, 1.356658], [103.96895, 1.35548], [103.968882, 1.35523], [103.968763, 1.354787], [103.968698, 1.35464], [103.96861, 1.354451], [103.968545, 1.354332], [103.968501, 1.354249], [103.968416, 1.354129], [103.968218, 1.353939], [103.968195, 1.353924], [103.968012, 1.353814], [103.967842, 1.353735], [103.967569, 1.353634], [103.967412, 1.353582], [103.967294, 1.353545], [103.966947, 1.353439], [103.966475, 1.353299], [103.966445, 1.35329], [103.965759, 1.353077], [103.965694, 1.353057], [103.965139, 1.352903], [103.964947, 1.352844], [103.964655, 1.352755], [103.964401, 1.352678], [103.963984, 1.352511], [103.963964, 1.352631], [103.963953, 1.35269], [103.96387, 1.352885], [103.963687, 1.353509], [103.963555, 1.354057], [103.963461, 1.354471], [103.963344, 1.354855], [103.963294, 1.355173], [103.963156, 1.355624], [103.962949, 1.356277], [103.962849, 1.356588], [103.962742, 1.356873], [103.96264, 1.357124], [103.962576, 1.357284], [103.962506, 1.357447], [103.962444, 1.357671], [103.962363, 1.357898], [103.962148, 1.358383], [103.96178, 1.359124], [103.961522, 1.359545], [103.961214, 1.360016], [103.960939, 1.360397], [103.960588, 1.360847], [103.960074, 1.361447], [103.959556, 1.361978], [103.95911, 1.362396], [103.95873, 1.362742], [103.958362, 1.363017], [103.958089, 1.363227], [103.95804, 1.36326], [103.957626, 1.363541], [103.956854, 1.364015], [103.956261, 1.364346], [103.955683, 1.364627], [103.954988, 1.364926], [103.954243, 1.365182], [103.953873, 1.365291], [103.953494, 1.365394], [103.95296, 1.365535], [103.951786, 1.365799], [103.950207, 1.36619], [103.948175, 1.366622], [103.946536, 1.367052], [103.945938, 1.367281], [103.945435, 1.367492], [103.944951, 1.367726], [103.944293, 1.368106], [103.943942, 1.368334], [103.943324, 1.368819], [103.942902, 1.369177], [103.942252, 1.36981], [103.94178, 1.370258], [103.941504, 1.370481], [103.941044, 1.370832], [103.940841, 1.371001], [103.940653, 1.371159], [103.940308, 1.371382], [103.93973, 1.371721], [103.938235, 1.372508], [103.937761, 1.372767], [103.93694, 1.373204], [103.936745, 1.373308], [103.936464, 1.37346], [103.935387, 1.37399], [103.935103, 1.374164], [103.934491, 1.374498], [103.933953, 1.374746], [103.933808, 1.374813], [103.932526, 1.375403], [103.931218, 1.376016], [103.930718, 1.376232], [103.929813, 1.376622], [103.929206, 1.376885], [103.928515, 1.377138], [103.92801, 1.377298], [103.927069, 1.377529], [103.926381, 1.377697], [103.925498, 1.377904], [103.925321, 1.377948], [103.923986, 1.378282], [103.923463, 1.378432], [103.922958, 1.378594], [103.922124, 1.378925], [103.921157, 1.379355], [103.920798, 1.379514], [103.920173, 1.379921], [103.919652, 1.38028], [103.919161, 1.380646], [103.918698, 1.381051], [103.918244, 1.381479], [103.917614, 1.382133], [103.917294, 1.382465], [103.917023, 1.382746], [103.916268, 1.38355], [103.915802, 1.3841], [103.915681, 1.384237], [103.915558, 1.384357], [103.915304, 1.38459], [103.914768, 1.385239], [103.913768, 1.386387], [103.913564, 1.386609], [103.913101, 1.387159], [103.912034, 1.388427], [103.911041, 1.389572], [103.910472, 1.390235], [103.909899, 1.390899], [103.909658, 1.391178], [103.909613, 1.39123], [103.909296, 1.391596], [103.908783, 1.392188], [103.908216, 1.392829], [103.908099, 1.392961], [103.907579, 1.39351], [103.907065, 1.394028], [103.906981, 1.394112], [103.906311, 1.394714], [103.906051, 1.394961], [103.905843, 1.395167], [103.905663, 1.395312], [103.905239, 1.395627], [103.904657, 1.396091], [103.903936, 1.396595], [103.903218, 1.397015], [103.902528, 1.397396], [103.902425, 1.397463], [103.901613, 1.39791], [103.900911, 1.398254], [103.900083, 1.398603], [103.898852, 1.39909], [103.898292, 1.399273], [103.897162, 1.399587], [103.896788, 1.399678], [103.894846, 1.400061], [103.894565, 1.400103], [103.893578, 1.400234], [103.892121, 1.400397], [103.89076, 1.400606], [103.88873, 1.400895], [103.887639, 1.401042], [103.886441, 1.401145], [103.885846, 1.401196], [103.884779, 1.401256], [103.883031, 1.401279], [103.881813, 1.401248], [103.879563, 1.401219], [103.879394, 1.401217], [103.87759, 1.401156], [103.876236, 1.401115], [103.875929, 1.401105], [103.875523, 1.401108], [103.875242, 1.401113], [103.87471, 1.401106], [103.872982, 1.401014], [103.872295, 1.401001], [103.871914, 1.400984], [103.871584, 1.400975], [103.870449, 1.400942], [103.870237, 1.400936], [103.86983, 1.400925], [103.869313, 1.400912], [103.86825, 1.400888], [103.864938, 1.400817], [103.863494, 1.40077], [103.862371, 1.400743], [103.861043, 1.400738], [103.860456, 1.400736], [103.859867, 1.400697], [103.859347, 1.400634], [103.859012, 1.400575], [103.858631, 1.400488], [103.857988, 1.400303], [103.8572, 1.39999], [103.856942, 1.399867], [103.856668, 1.399733], [103.856488, 1.399648], [103.856265, 1.399527], [103.855869, 1.399294], [103.855754, 1.399213], [103.855444, 1.39899], [103.855087, 1.398681], [103.854796, 1.398396], [103.854481, 1.398081], [103.854095, 1.397715], [103.853976, 1.397578], [103.853834, 1.397418], [103.853586, 1.397161], [103.853258, 1.396873], [103.852854, 1.396664], [103.852462, 1.396504], [103.85208, 1.396428], [103.851695, 1.39637], [103.850213, 1.39629], [103.849518, 1.396285], [103.848912, 1.396328], [103.848541, 1.396357], [103.848172, 1.396419], [103.846987, 1.396486], [103.846646, 1.396497], [103.845479, 1.396512], [103.844378, 1.396463], [103.843277, 1.396368], [103.842527, 1.396284], [103.841778, 1.396148], [103.841001, 1.395994], [103.840225, 1.395817], [103.83989, 1.395732], [103.838706, 1.395413], [103.838208, 1.395297], [103.837097, 1.395018], [103.836995, 1.394991], [103.83585, 1.394714], [103.831781, 1.393691], [103.830953, 1.393447], [103.830643, 1.39337], [103.830439, 1.393315], [103.828645, 1.392981], [103.827894, 1.392899], [103.826533, 1.392823], [103.825, 1.392854], [103.824616, 1.3929], [103.824278, 1.392935], [103.823767, 1.393001], [103.822623, 1.393191], [103.821953, 1.393354], [103.821447, 1.393492], [103.820679, 1.393707], [103.819933, 1.393935], [103.818977, 1.394215], [103.81881, 1.394265], [103.818091, 1.394478], [103.817682, 1.394591], [103.817295, 1.394695], [103.816948, 1.394802], [103.816624, 1.394922], [103.816288, 1.395056], [103.81589, 1.395281], [103.815696, 1.395409], [103.815351, 1.395674], [103.815033, 1.395957], [103.814878, 1.396107], [103.814586, 1.396456], [103.814187, 1.397047], [103.813852, 1.397707], [103.813524, 1.398666], [103.813151, 1.399725], [103.812823, 1.400536], [103.812389, 1.401445], [103.812245, 1.401706], [103.812042, 1.402084], [103.811881, 1.402357], [103.811331, 1.403253], [103.810836, 1.404019], [103.810383, 1.404594], [103.809795, 1.405286], [103.8096, 1.405492], [103.809383, 1.405776], [103.808664, 1.406953], [103.808568, 1.407127], [103.807876, 1.408721], [103.807729, 1.409072], [103.807571, 1.409508], [103.806486, 1.412336], [103.805946, 1.413655], [103.805566, 1.414379], [103.805229, 1.414849], [103.80463, 1.415553], [103.804129, 1.416036], [103.803529, 1.416546], [103.803312, 1.416711], [103.80317, 1.416835], [103.802379, 1.417532], [103.800561, 1.419025], [103.798818, 1.42028], [103.798452, 1.420503], [103.797743, 1.420938], [103.797274, 1.421178], [103.797195, 1.421178], [103.797159, 1.421188], [103.797105, 1.421208], [103.797036, 1.421241], [103.796959, 1.421281], [103.796653, 1.421409], [103.796433, 1.421506], [103.796068, 1.421667], [103.795592, 1.421854], [103.79552, 1.421888], [103.795378, 1.421951], [103.795278, 1.422016], [103.795173, 1.422157], [103.795123, 1.422262], [103.795093, 1.422396], [103.795101, 1.422575], [103.795148, 1.422676], [103.795909, 1.423586], [103.796284, 1.424011], [103.797271, 1.425158], [103.798166, 1.426199], [103.798447, 1.42653], [103.798528, 1.426624], [103.798561, 1.426664], [103.798564, 1.426667], [103.798569, 1.426672], [103.798681, 1.426804], [103.798714, 1.426841], [103.79878, 1.426919], [103.798835, 1.426981], [103.798948, 1.427111], [103.798999, 1.427169], [103.799397, 1.427633], [103.799839, 1.428142], [103.800004, 1.428333], [103.800103, 1.428447], [103.800746, 1.429185], [103.801547, 1.430141], [103.802757, 1.431553], [103.803209, 1.43208], [103.80324, 1.432192], [103.803238, 1.432277], [103.803232, 1.432353], [103.803176, 1.43248], [103.803062, 1.432565], [103.802584, 1.432835], [103.802231, 1.432982], [103.802122, 1.433006], [103.801835, 1.433015], [103.801639, 1.433021], [103.80161, 1.43302], [103.801516, 1.433019], [103.801522, 1.433171], [103.801529, 1.433234], [103.801553, 1.433476], [103.801555, 1.433557], [103.801556, 1.433615], [103.801579, 1.43386], [103.801635, 1.434116], [103.801799, 1.434479], [103.801875, 1.434596], [103.80219, 1.434966], [103.80231, 1.435102], [103.802359, 1.435161], [103.802605, 1.435458], [103.802704, 1.435576], [103.802728, 1.435603], [103.802852, 1.435748], [103.803137, 1.436119], [103.803203, 1.436206], [103.803248, 1.436262], [103.803354, 1.436412], [103.803425, 1.436529], [103.803492, 1.436662], [103.803558, 1.436906], [103.803583, 1.437034], [103.803576, 1.437224], [103.803572, 1.437351], [103.803557, 1.437476], [103.803513, 1.437639], [103.80333, 1.438027], [103.803271, 1.438172], [103.803208, 1.43831], [103.803019, 1.438721], [103.803003, 1.438755], [103.802988, 1.438787], [103.802945, 1.438885], [103.802711, 1.439409], [103.802562, 1.439749], [103.802481, 1.439948], [103.802357, 1.440244], [103.802263, 1.440465], [103.80219, 1.440625], [103.802162, 1.440689], [103.802125, 1.440778], [103.802048, 1.440962], [103.801977, 1.441116], [103.801878, 1.441194], [103.801808, 1.441225], [103.801766, 1.441234], [103.801629, 1.44126], [103.801096, 1.440982], [103.800854, 1.440875]]

route_latlng1 = [(lat, lon) for lon, lat in route_latlng1]

In [37]:
route_latlng

[(1.357148, 103.818883),
 (1.357148, 103.818883),
 (1.357162, 103.818935),
 (1.357189, 103.818981),
 (1.357223, 103.819012),
 (1.35729, 103.819058),
 (1.357362, 103.81909),
 (1.357968, 103.819339),
 (1.358546, 103.819583),
 (1.358627, 103.819625),
 (1.358757, 103.819728),
 (1.359022, 103.819982),
 (1.359227, 103.820179),
 (1.359462, 103.820387),
 (1.359757, 103.820582),
 (1.360217, 103.82077),
 (1.360514, 103.820927),
 (1.360783, 103.821092),
 (1.360879, 103.821166),
 (1.360953, 103.821265),
 (1.360916, 103.821533),
 (1.360924, 103.821745),
 (1.360973, 103.821974),
 (1.361, 103.822097),
 (1.361025, 103.822213),
 (1.361029, 103.822306),
 (1.36102, 103.822405),
 (1.360991, 103.822617),
 (1.360947, 103.822871),
 (1.360905, 103.823165),
 (1.360854, 103.823466),
 (1.360848, 103.823511),
 (1.360822, 103.823684),
 (1.360812, 103.823865),
 (1.360831, 103.82403),
 (1.360871, 103.824181),
 (1.360921, 103.824313),
 (1.361136, 103.82472),
 (1.361207, 103.824838),
 (1.361271, 103.824978),
 (1.36134